[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ffserro/MVP/blob/master/mvp.ipynb)


# Regressão Linear para Series Temporais - Planejamento dos dispêndios de alimentação de militares da Marinha do Brasil

## Contextualização



<div align="justify" font-size="40px">
O planejamento eficiente dos recursos logísticos é um dos pilares para a manutenção da prontidão e da capacidade operacional das Forças Armadas. Entre os diversos insumos estratégicos, a alimentação das organizações militares desempenha papel central, tanto no aspecto orçamentário quanto no suporte direto às atividades diárias. Na Marinha do Brasil, a gestão dos estoques e dos gastos com gêneros alimentícios envolve múltiplos órgãos e abrange um volume expressivo de transações financeiras e contábeis, tornando-se um processo complexo e suscetível a variações sazonais, econômicas e administrativas.

Neste cenário, prever com maior precisão os custos relacionados ao consumo de alimentos é fundamental para otimizar a alocação de recursos públicos, reduzir desperdícios, evitar rupturas de estoque e aumentar a eficiência do planejamento orçamentário. Tradicionalmente, esse processo é conduzido por meio de análises históricas e técnicas de planejamento administrativo. No entanto, tais abordagens muitas vezes não capturam adequadamente os padrões temporais e as variáveis externas que influenciam os gastos.

A ciência de dados, e em particular as técnicas de modelagem de séries temporais, surge como uma alternativa poderosa para aprimorar esse processo decisório. Modelos como SARIMA, Prophet, XGBoost e LSTM permitem identificar tendências, sazonalidades e anomalias nos dados, possibilitando não apenas previsões mais robustas, mas também a geração de insights que subsidiam políticas de abastecimento e aquisição.

Assim, o presente trabalho propõe a aplicação de técnicas de análise e previsão de séries temporais sobre os dados históricos de consumo de alimentos da Marinha do Brasil, com o objetivo de estimar os custos futuros e explorar padrões relevantes que possam apoiar o processo de gestão logística e orçamentária. A relevância deste estudo reside não apenas no ganho potencial de eficiência administrativa, mas também na contribuição para a transparência, a racionalização do gasto público e a modernização da gestão de suprimentos em instituições estratégicas para o país.
</div>

## Glossário


* Municiamento
* Rancho
* Etapa
* Comensal
* Série Temporal
* Tendência
* Sazonalidade
* Estacionariedade


## Trabalho

In [24]:
#@title Download dos dados
!git clone 'https://github.com/ffserro/MVP.git'
!pip install -r '/content/MVP/requirements.txt' > '/content/pip_log.txt'
!if grep -iq "downloading" '/content/pip_log.txt';then python -c "import os;print('Por favor, reinicie a sessão e execute novamente.');os.kill(os.getpid(), 9)"; else python -c "print('Vamos começar!')"; fi

fatal: destination path 'MVP' already exists and is not an empty directory.
Vamos começar!


In [25]:
#@title Import de bibliotecas
from glob import glob

from datetime import date as dt, timedelta as td

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from geopy.geocoders import Nominatim

import pmdarima as pm
from prophet import Prophet
from xgboost import XGBRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [26]:
mmm = pd.DataFrame()
mmm = pd.concat([mmm]+[pd.read_excel(d, parse_dates=[['ano', 'mes']]) for d in glob('/content/MVP/dados/mmm/*')]).reset_index(drop=True)

etapas = pd.DataFrame()
etapas = pd.concat([etapas]+[pd.read_excel(d, parse_dates=[['ano', 'mes']]) for d in glob('/content/MVP/dados/etapas/*')]).reset_index(drop=True).rename(columns={'uasg':'codigo'})

centralizadas = pd.read_csv('/content/MVP/dados/REL_OM_CENTRALIZADA_202509051127.csv', parse_dates=['DATA_INICIO', 'DATA_FIM'])
centralizadas = centralizadas[~(centralizadas.TIPO_CENTRALIZACAO_ID==4)]
centralizadas.drop(columns=['CONTATO', 'TELEFONE', 'CRIACAO', 'MODIFICACAO', 'TIPO_CENTRALIZACAO_ID', 'GESTORIA_ID'], inplace=True)
for codigo in centralizadas.OM_CENTRALIZADA_ID.unique():
  if centralizadas.OM_CENTRALIZADA_ID.value_counts().loc[codigo] > 1:
    centralizadas.loc[centralizadas[centralizadas.OM_CENTRALIZADA_ID==codigo].sort_values(by='DATA_INICIO').index, 'DATA_FIM'] = centralizadas[centralizadas.OM_CENTRALIZADA_ID==codigo].sort_values(by='DATA_INICIO').DATA_INICIO.shift(-1)
centralizadas.DATA_FIM.fillna(dt(2026, 1,1), inplace=True) # preenchendo os Not a Time com uma data no futuro. (É o caso das que não tem data fim)

om_info = pd.read_csv('/content/MVP/dados/OM_202509051130.csv', delimiter=';', encoding='latin-1')

In [27]:
om_info.head()

,DN_ID,AREA_ID,ID,CODIGO,NOME,TIPO,INDICATIVO_NAVAL,SIGLA,COMIMSUP,ODS,...,COD_SQ_LOCAL,TIPO_CONEXAO,CNPJ,ENDERECO,BAIRRO,CIDADE,UF,CEP,TELEFONE,SUB_DN_ID
0,7.0,NaN,10323,20100,MISSÃO NAVAL BRASILEIRA NA NAMÍBIA,T,MNBNAM,MNBN,NaN,NaN,...,323.0,C,"3,95E+11","EMBASSY OF BRAZIL - 52, BISMARCK STREET, PO BO...",WINDHOEK WEST,WINDHOEK - KHOMAS,NaN,9000.0,(00XX264) 61-255297,7.0
1,1.0,2.0,10216,89145,NAVIO-PATRULHA GUAPORÉ,N,NPAORE,NPaGuapore,NaN,89000.0,...,216.0,C,"3,95E+11","AVENIDA CIDADE DE SANTOS S/N, CAIS DA MARINHA",MACUCO,SANTOS,SP,11015050.0,NaN,1.0
2,8.0,2.0,13681,89101,COMANDO DO GRUPAMENTO DE PATRULHA NAVAL DO SUL...,T,GPNSSE,COMGPTPATNAVSSE,NaN,NaN,...,599.0,C,"3,95E+11","AVENIDA CIDADE DE SANTOS, S/N, CAIS DA MARINHA",MACUCO,SANTOS,SP,11015050.0,NaN,27.0
3,8.0,2.0,14459,89015,Navio-Patrulha Maracanã,N,NPAMRC,NPaMaracana,NaN,NaN,...,459.0,C,"3,95E+11","Avenida Cidade de Santos, S/N°",Macuco,Santos,SP,11015050.0,8814-2004,27.0
4,1.0,2.0,10100,89144,NAVIO-PATRULHA GUAJARÁ,N,NPAGUA,NPaGuajara,NaN,89000.0,...,100.0,C,"3,95E+11","AV.ISMAEL COELHO DE SOUZA, S/N",MACUCO,SANTOS,SP,11015315.0,"(0XX21) 3849-6662, 3849-6357 Retelma: 8126-6662",1.0


In [28]:
om_info.CEP

,CEP
0,9000.0
1,11015050.0
2,11015050.0
3,11015050.0
4,11015315.0
...,...
458,5508030.0
459,NaN
460,NaN
461,NaN


In [29]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Avenida Rio Branco 1, Rio de Janeiro'}
r = requests.get(url, params=params)

In [30]:
r.json()

{'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account',
 'results': [],
 'status': 'REQUEST_DENIED'}

In [31]:
om_info.ENDERECO

,ENDERECO
0,"EMBASSY OF BRAZIL - 52, BISMARCK STREET, PO BO..."
1,"AVENIDA CIDADE DE SANTOS S/N, CAIS DA MARINHA"
2,"AVENIDA CIDADE DE SANTOS, S/N, CAIS DA MARINHA"
3,"Avenida Cidade de Santos, S/N°"
4,"AV.ISMAEL COELHO DE SOUZA, S/N"
...,...
458,"Av. Prof. Mello de Moraes, 2231 BL E-Depto Eng..."
459,"170, UPPER RICHMOND ROAD - PUTNEY -LONDON - SW..."
460,"ESTRADA DAS LARANJEIRAS, 144"
461,170 UPPER RICHMOND ROAD - 3º ANDAR


In [32]:
# Dicionário para converter do código interno para número da uasg
tradutor_id_codigo = {row.ID: row.CODIGO for _, row in om_info.iterrows()}

centralizadas[['OM_CENTRALIZADA_ID', 'OM_CENTRALIZADORA_ID']] = centralizadas[['OM_CENTRALIZADA_ID', 'OM_CENTRALIZADORA_ID']].replace(tradutor_id_codigo)

In [33]:
centralizadas['distancia'] = centralizadas['OM_CENTRALIZADA_ID'].map(om_info.set_index('CODIGO')['CEP']) - centralizadas['OM_CENTRALIZADORA_ID'].map(om_info.set_index('CODIGO')['CEP'])

In [34]:
centralizadas['apoiadora'] = centralizadas['OM_CENTRALIZADORA_ID'].map(om_info.set_index('CODIGO')['INDICATIVO_NAVAL'])
centralizadas['apoiada'] = centralizadas['OM_CENTRALIZADA_ID'].map(om_info.set_index('CODIGO')['INDICATIVO_NAVAL'])

In [35]:
centralizadas.sort_values(by='distancia')

,OM_CENTRALIZADA_ID,OM_CENTRALIZADORA_ID,DATA_INICIO,DATA_FIM,distancia,apoiadora,apoiada
303,20300,20301,2018-06-01,2024-03-01 00:00:00,-16423090.0,INSPEQ,DRGNUC
641,42020,42030,2022-10-01,2022-10-01 00:00:00,-13052000.0,ARAMAR,DDNUCM
71,91902,53000,2009-04-01,2023-06-01 00:00:00,-4889700.0,IEMAPM,CASOPE
691,40030,91540,2024-03-01,2026-01-01,-3735640.0,BASNUC,INDUST
90,65700,51212,2004-01-01,2004-01-01 00:00:00,-3528847.0,BASHID,CMASSM
...,...,...,...,...,...,...,...
606,12728,12728,2022-01-01,2026-01-01,NaN,NaN,NaN
607,12703,12703,2022-01-01,2026-01-01,NaN,NaN,NaN
608,12704,12704,2022-01-01,2026-01-01,NaN,NaN,NaN
609,12705,12705,2022-01-01,2026-01-01,NaN,NaN,NaN


In [36]:
# Filtrando apenas as OM que possuem dados de todo o período observado (80 meses)
mmm = mmm[(mmm.codigo.isin((mmm.codigo.value_counts()==80).index)) & ~(mmm.nome.str.contains('TREINA'))]

In [37]:
etapas[etapas.codigo.isin(mmm.codigo.unique())]

,ano_mes,codigo,codigo_etapa,quantidade
1,2019-01-01,10100,101,1316.0
2,2019-01-01,10100,103,573.0
3,2019-01-01,10100,105,420.0
4,2019-01-01,10100,414,449.0
5,2019-01-01,10100,620,1.0
...,...,...,...,...
105504,2025-07-01,95400,629,18351.0
105505,2025-07-01,95500,101,7260.0
105506,2025-07-01,95500,103,5262.0
105507,2025-07-01,95500,604,62.0


In [38]:
centralizadas[centralizadas.OM_CENTRALIZADA_ID==14000]

,OM_CENTRALIZADA_ID,OM_CENTRALIZADORA_ID,DATA_INICIO,DATA_FIM,distancia,apoiadora,apoiada
82,14000,87000,2009-05-01,2022-05-01 00:00:00,0.0,SETDIS,CIMBRA
682,14000,14000,2022-05-01,2024-03-01 00:00:00,0.0,CIMBRA,CIMBRA
769,14000,87010,2024-03-01,2026-01-01,0.0,CITBRA,CIMBRA


In [39]:
# etapas['centralizadoras'] = [centralizadas[(centralizadas.DATA_INICIO<=row.ano_mes) & (centralizadas.DATA_FIM>row.ano_mes) & (centralizadas.OM_CENTRALIZADA_ID==row.codigo)].OM_CENTRALIZADORA_ID.values for _, row in etapas.iterrows()]

merged = pd.merge(etapas, centralizadas, how='left',
                  left_on='codigo', right_on='OM_CENTRALIZADA_ID')

# Filtrando as linhas onde as datas atendem as condições (usando a máscara booleana)
merged = merged[(merged['DATA_INICIO'] <= merged['ano_mes']) &
                (merged['DATA_FIM'] > merged['ano_mes'])]

# Agora, podemos simplesmente atribuir a coluna 'OM_CENTRALIZADORA_ID'
etapas['centralizadoras'] = merged.groupby('codigo')['OM_CENTRALIZADORA_ID'].first().values

del merged


ValueError: Length of values (351) does not match length of index (105516)

In [ ]:
etapas

In [ ]:
centralizadas[centralizadas.OM_CENTRALIZADA_ID==11100]

In [ ]:
tradutor_id_codigo

In [ ]:
centralizadas

In [ ]:
etapas

In [ ]:
mmm_custo_unitario = pd.merge(left=mmm, right=etapas.groupby(['ano', 'mes', 'codigo']).quantidade.sum().reset_index(), how='inner', on=['codigo', 'ano', 'mes'])[['ano', 'mes', 'codigo', 'nome', 'totais_balanco_paiol_despesa', 'quantidade']]

In [ ]:
mmm_custo_unitario['custo_unitario'] = mmm_custo_unitario.totais_balanco_paiol_despesa / mmm_custo_unitario.quantidade

In [ ]:
mmm_custo_unitario.sort_values(by='custo_unitario').tail(50)

In [ ]:
mmm_custo_unitario.custo_unitario.describe()

In [ ]:
etapas[etapas.codigo_etapa < 600].groupby(['ano', 'mes', 'codigo']).quantidade.sum().reset_index()

In [ ]:
mmm_marinha = mmm.groupby(['ano', 'mes'])[[col for col in mmm.columns if col not in ['ano', 'mes', 'codigo', 'nome']]].sum().reset_index()

In [ ]:
mmm_marinha['mes'] = [''.join([str(i[0]), '_', str(i[1])]) for i in zip(mmm_marinha.mes.values, mmm_marinha.ano.values)]
mmm_marinha.drop(columns=['ano'], inplace=True)
mmm_marinha = mmm_marinha.iloc[:-2]

In [ ]:
mmm_etapas = pd.merge(left=mmm, right=etapas, how='inner', on=['ano', 'mes', 'codigo'])

In [ ]:
mmm_etapas[mmm_etapas.codigo_etapa.isin([103, 105])][['ano', 'mes', 'nome', 'codigo_etapa', 'quantidade']]

In [ ]:
pd.DataFrame({
    'diferenca': mmm_receita_despesa.despesa_autorizada_global - (mmm_receita_despesa.generos_consumidos + mmm_receita_despesa.vale_extra),
    'sobra_licita': mmm_receita_despesa.sobra_licita
})

In [ ]:
da_marinha = da.groupby(['mes', 'ano']).despesa_autorizada_global.sum().reset_index()
da_marinha['mes'] = ['_'.join(['{:02d}'.format(i[0]), str(i[1])]) for i in zip(da_marinha.mes.values, da_marinha.ano.values)]
da_marinha = da_marinha.sort_values(by=['ano', 'mes']).iloc[:-2]
fig = px.line(
    da_marinha,
    x = 'mes',
    y = 'despesa_autorizada_global',
    title='Despesa Autorizada global'
)

fig.show()

In [ ]:
def grafico_base(titulo):
    return px.line(
        mmm_marinha,
        x = 'mes',
        y = 'totais_balanco_paiol_despesa',
        labels = {
            'mes': 'Mês e ano',
            'totais_balanco_paiol_despesa': 'Totais das despesas'
        },
        title = titulo
    )

fig = grafico_base('Gastos com alimentação dos últimos cinco anos')

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Teste de estacionariedade

result = adfuller(mmm_marinha.totais_balanco_paiol_despesa)
print(f'ADF: {result[0]}, p-valor: {result[1]}')

In [ ]:
naive_forecast = mmm_marinha.totais_balanco_paiol_despesa.shift(1)
mae_naive = mean_absolute_error(mmm_marinha.totais_balanco_paiol_despesa.iloc[1:], naive_forecast.iloc[1:])
print('Baseline Naïve MAE:', mae_naive)

In [ ]:
# SARIMA

train = mmm_marinha.iloc[:-12, :]
test = mmm_marinha.iloc[-12:, :]

model_auto = pm.auto_arima(train.totais_balanco_paiol_despesa, seasonal=True, m=12, stepwise=True, trace=True)
print(model_auto.summary())

In [ ]:
mmm

In [ ]:
sarima_model = SARIMAX(train.totais_balanco_paiol_despesa,
                       order=model_auto.order,
                       seasonal_order=model_auto.seasonal_order)

sarima_fit = sarima_model.fit(disp=False)

forecast_sarima = sarima_fit.get_forecast(steps=12)
pred_sarima = forecast_sarima.predicted_mean

mae_sarima = mean_absolute_error(test.totais_balanco_paiol_despesa, pred_sarima)
print('SARIMA MAE:', mae_sarima)



In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo SARIMA')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_sarima
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Previsão com o Prophet

prophet_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
prophet_df.columns = ['ds', 'y']
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'], format='%m_%Y')

model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.fit(prophet_df)

future = model_prophet.make_future_dataframe(periods=12, freq='M')
forecast = model_prophet.predict(future)

forecast_test = forecast.set_index('ds').loc[pd.to_datetime(test.mes, format='%m_%Y')]
mae_prophet = mean_absolute_error(test['totais_balanco_paiol_despesa'], forecast_test['yhat'])
print('Prophet MAE', mae_prophet)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo Prophet')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = forecast_test.yhat
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Regressão com XGBoost para séries temporais

xg_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
xg_df['ano'] = xg_df['mes'].apply(lambda x: x.split('_')[-1]).astype(int)
xg_df['mes'] = xg_df['mes'].apply(lambda x: x.split('_')[0]).astype(int)
xg_df['lag1'] = xg_df['totais_balanco_paiol_despesa'].shift(1)
xg_df['lag3'] = xg_df['totais_balanco_paiol_despesa'].shift(3)
xg_df['lag6'] = xg_df['totais_balanco_paiol_despesa'].shift(6)
xg_df['rolling3'] = xg_df['totais_balanco_paiol_despesa'].rolling(3).mean()
xg_df['rolling6'] = xg_df['totais_balanco_paiol_despesa'].rolling(6).mean()

xg_df = xg_df.dropna().reset_index(drop=True)

In [ ]:
train_xg = xg_df.iloc[:-12]
test_xg = xg_df.iloc[-12:]

X_train = train_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_train = train_xg['totais_balanco_paiol_despesa']
X_test = test_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_test = test_xg['totais_balanco_paiol_despesa']

xgb = XGBRegressor(
    n_estimators = 300,
    learning_rate = 0.05,
    max_depth = 5,
    subsample = 0.8,
    colsample_bytree = 0.8,
    random_state = 42
)

xgb.fit(X_train, y_train)

pred_xgb = xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, pred_xgb)

print("XGBoost MAE:", mae_xgb)


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo XGBoost')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_xgb
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
hw_model = ExponentialSmoothing(
    train['totais_balanco_paiol_despesa'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()

pred_hw = hw_model.forecast(12)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo ExponentialSmoothing')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_hw
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
despesas_scaled = scaler.fit_transform(mmm_marinha.totais_balanco_paiol_despesa.values.reshape(-1, 1))

def create_sequences(data, window=12):
    X, y = [], []
    for i in range(len(data)- window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(despesas_scaled)


split = len(X) - 12
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('Shape treino:', X_train.shape, y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='tanh', return_sequences=True, input_shape=(12, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae')

history = model.fit(
    X_train, y_train,
    epochs = 200,
    batch_size = 4,
    validation_split = 0.1,
    verbose = 1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv)
print('LSTM MAE:', mae_lstm)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.reshape(1, -1)[0]
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
def create_sequences_multistep(data, window=12, horizon=12):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window:i+window+horizon].flatten())
    return np.array(X), np.array(y)

window = 12
horizon = 12
X, y = create_sequences_multistep(despesas_scaled, window, horizon)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('X_train shape:', X_train.shape)
print('y_train, shape:', y_train.shape)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation="tanh", return_sequences=True, input_shape=(window, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation="tanh"))
model.add(Dense(horizon))  # saída com 12 valores (multi-step)

model.compile(optimizer="adam", loss="mse")

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=4,
    validation_split=0.1,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm_multi = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
print('LSTM Multi-step MAE:', mae_lstm_multi)

In [ ]:
last_input_dates = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-(window + horizon):-horizon]
future_dates = pd.date_range(start = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-horizon], periods=horizon, freq='M')

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM Multi-step')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.flatten()
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
teste = ['A', 'B', 'C']

In [ ]:
model = 'LSTM' #@param ["SARIMAX", "prophet", "XGBoost", "ExponentialSmothing", "LSTM"]
test = 'A' #@param

print(f"Selected model: {modelo}")
print(f'Selected test : {test}')

In [ ]:
mmm['mes_ano'] = mmm.mes.astype(str) + '_' + mmm.ano.astype(str)

In [ ]:
# !playwright install chromium

In [ ]:
# from playwright.async_api import async_playwright
# import time

In [ ]:
# with async_playwright() as p:
#     browser = p.chromium.launch(headless=False)
#     page = browser.new_page()

#     page.goto('https://www.maps.google.com')

#     page.locator('xpath=/html/body/div[1]/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/form/input').fill('Avenida Rio Branco 1, Rio de Janeiro')
#     page.keyboard.press('Enter')
#     time.sleep(1)
#     print(page.url())

In [ ]:
# from playwright.async_api import async_playwright
# import asyncio

# async def get_latlong(endereco):
#   async with async_playwright() as p:
#     browser = await p.chromium.launch()
#     page = await browser.new_page()
#     await page.goto('https://www.google.com/maps')

#     await page.locator('#searchboxinput').fill('Avenida Rio Branco 1, Rio de Janeiro')
#     await page.keyboard.press('Enter')
#     await page.wait_for_timeout(5000)
#     time.sleep(1)
#     return page.url.split('@')[1].split(',')[:2]

In [ ]:
# enderecos = {}

# for endereco in om_info.ENDERECO.unique():
#   enderecos[endereco] = await get_latlong(endereco)

In [ ]:
enderecos

Now that the data has been processed and the necessary transformations have been applied, we can proceed with evaluating the time series forecasting models. The notebook currently includes code for SARIMA, Prophet, XGBoost, Exponential Smoothing, and LSTM.

**Next Steps:**

1.  **Evaluate Models:** Run the code cells for each model to generate predictions.
2.  **Compare Performance:** Calculate relevant evaluation metrics (e.g., Mean Absolute Error - MAE) for each model and compare their performance.
3.  **Visualize Results:** Plot the predictions of each model against the actual data to visually assess their accuracy.
4.  **Select Best Model:** Based on the evaluation metrics and visualizations, select the most suitable model for forecasting the food expenditures.
5.  **Further Analysis (Optional):** Explore potential improvements or alternative models if needed.